In [148]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
path_final = 'data_final/'

In [149]:
df_test1 = pd.read_csv(path_final+"test_final_0.csv")
df_test2 = pd.read_csv(path_final+"test_final_5.csv")
df_test3 = pd.read_csv(path_final+"test_final_10.csv")
df_test4 = pd.read_csv(path_final+"test_final_15.csv")
df_train1 = pd.read_csv(path_final+"train_final_0.csv")
df_train2 = pd.read_csv(path_final+"train_final_5.csv")
df_train3 = pd.read_csv(path_final+"train_final_10.csv")
df_train4 = pd.read_csv(path_final+"train_final_15.csv")

In [150]:
df_train_list = [df_train1,df_train2, df_train3, df_train4]
df_test_list = [df_test1, df_test2, df_test3, df_test4]

In [151]:
random.shuffle(df_train_list)
random.shuffle(df_test_list)

In [152]:
df_train = pd.concat(df_train_list)

In [154]:
df_test = pd.concat(df_test_list)

In [155]:
df_train.count()

time_start        51825
volume            51825
direction         51825
tollgate_id       51825
time_window       51825
weekday           51825
weekend           51825
date              51825
hour              51825
minute            51825
holiday           51825
am_pm             51825
pressure          16680
sea_pressure      16680
wind_direction    16680
wind_speed        16680
temperature       16680
rel_humidity      16680
precipitation     16680
dtype: int64

In [156]:
df_test.head()

,time_start,volume,direction,tollgate_id,time_window,weekday,weekend,date,hour,minute,holiday,am_pm,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,2016-10-25 00:15:00,0,0,1,"[00:15:00,00:35:00)",2,0,2016-10-25,0,15,False,1,1010.6,1015.6,356.0,3.4,20.2,86.0,0.0
1,2016-10-25 00:35:00,0,0,1,"[00:35:00,00:55:00)",2,0,2016-10-25,0,35,False,1,1010.6,1015.6,356.0,3.4,20.2,86.0,0.0
2,2016-10-25 00:55:00,0,0,1,"[00:55:00,01:15:00)",2,0,2016-10-25,0,55,False,1,1010.6,1015.6,356.0,3.4,20.2,86.0,0.0
3,2016-10-25 01:15:00,0,0,1,"[01:15:00,01:35:00)",2,0,2016-10-25,1,15,False,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-10-25 01:35:00,0,0,1,"[01:35:00,01:55:00)",2,0,2016-10-25,1,35,False,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
df_remove = df_test.loc[(df_test['volume'] == 0)]

df_test = df_test.drop(df_remove.index)

In [158]:
def feature_format():
    #pd_volume_train = pd_volume_train.set_index(['time'])
    #pd_volume_test = pd_volume_test.set_index(['time'])
    #volume_train = v_train.groupby(['time_window','tollgate_id','direction','date', 'hour']).size().reset_index().rename(columns = {0:'volume'})
    #volume_test = v_test.groupby(['time_window','tollgate_id','direction','date', 'hour']).size().reset_index().rename(columns = {0:'volume'})
    #print(volume_train)                
    #x = pd.Series(df_train['time_window'].unique())
    #s = pd.Series(range(len(x)),index = x.values)
    #df_train['window_n'] = df_train['time_window'].map(s)
    #df_test['window_n'] = df_test['time_window'].map(s)
#        print vol_test.tail()
    #volume_train['weekday'] = v_train['weekday']
    #volume_test['weekday'] = v_test['weekday']
    
    feature_train = df_train.drop('volume', axis = 1)
    feature_test = df_test.drop('volume',axis = 1)
    values_train = df_train['volume'].values
    values_test = df_test['volume'].values
    
    return feature_train, feature_test, values_train, values_test

In [159]:
feature_train, feature_test, values_train, values_test = feature_format()
feature_train.head()

,time_start,direction,tollgate_id,time_window,weekday,weekend,date,hour,minute,holiday,am_pm,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,2016-09-19 00:05:00,0,1,"[00:05:00,00:25:00)",1,0,2016-09-19,0,5,False,1,1008.2,1013.2,329.0,2.8,22.2,76.0,0.0
1,2016-09-19 00:25:00,0,1,"[00:25:00,00:45:00)",1,0,2016-09-19,0,25,False,1,1008.2,1013.2,329.0,2.8,22.2,76.0,0.0
2,2016-09-19 00:45:00,0,1,"[00:45:00,01:05:00)",1,0,2016-09-19,0,45,False,1,1008.2,1013.2,329.0,2.8,22.2,76.0,0.0
3,2016-09-19 01:05:00,0,1,"[01:05:00,01:25:00)",1,0,2016-09-19,1,5,False,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-09-19 01:25:00,0,1,"[01:25:00,01:45:00)",1,0,2016-09-19,1,25,False,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [160]:
def feature_transform_split(data):
    data["precipitation"] = data[["precipitation"]].fillna(value=0)
    data["rel_humidity"] = data[["rel_humidity"]].fillna(value=50)

    data["precipitation"] = data["precipitation"].apply(lambda x: x > 0)
    data["rel_humidity"] = data["rel_humidity"].apply(lambda x: x > 90)

    data = data.drop("precipitation", axis=1)
    return data

In [138]:
def feature_transform_split_complete(key, data):
    # data = remove_exception(data)

    data["precipitation"] = data[["precipitation"]].fillna(value=0)
    data["rel_humidity"] = data[["rel_humidity"]].fillna(value=50)

    data["precipitation"] = data["precipitation"].apply(lambda x: x > 0)
    data["rel_humidity"] = data["rel_humidity"].apply(lambda x: x > 90)

    data = data.drop("precipitation", axis=1)
    # data = data.drop("rel_humidity", axis= 1)




    # data["sum"] = data["0"] + data["1"] + data["2"] + data["3"] + data["4"] + data["5"]

    #data = pd.concat([data, pd.get_dummies(data['period_num'])], axis=1)
    #data = data.drop("period_num", axis=1)

    data = pd.concat([data, pd.get_dummies(data['holiday'])], axis=1)
    data = data.drop("holiday", axis=1)
    #
    # data = pd.concat([data, pd.get_dummies(data['first_last_workday'])], axis=1)
    #data = data.drop("first_last_workday", axis=1)

    #data = data.drop("day_of_week", axis=1)

    if (key == 1):
        data = pd.concat([data, pd.get_dummies(data['tollgate_id'])], axis=1)
        # data["tollgate_id1"] = data['tollgate_id']
        data["direction1"] = data['direction']
    return data

In [162]:
feature_train = feature_transform_split(feature_train)

In [161]:
feature_test = feature_transform_split(feature_test)

In [141]:
feature_train.head()

,time_start,direction,tollgate_id,time_window,weekday,weekend,date,hour,minute,holiday,am_pm,rel_humidity
0,2016-09-19 00:05:00,0,1,"[00:05:00,00:25:00)",1,0,2016-09-19,0,5,False,1,False
1,2016-09-19 00:25:00,0,1,"[00:25:00,00:45:00)",1,0,2016-09-19,0,25,False,1,False
2,2016-09-19 00:45:00,0,1,"[00:45:00,01:05:00)",1,0,2016-09-19,0,45,False,1,False
3,2016-09-19 01:05:00,0,1,"[01:05:00,01:25:00)",1,0,2016-09-19,1,5,False,1,False
4,2016-09-19 01:25:00,0,1,"[01:25:00,01:45:00)",1,0,2016-09-19,1,25,False,1,False


In [142]:
regressor_cubic = RandomForestRegressor(n_estimators=500, max_features='sqrt', random_state=10, oob_score=True)

In [143]:
#x = pd.Series(df_train['time_window'].unique())
#s = pd.Series(range(len(x)),index = x.values)
#df_train['window_n'] = df_train['time_window'].map(s)
#df_test['window_n'] = df_test['time_window'].map(s)
df_train_grouped = df_train.groupby(["tollgate_id", "direction"])
df_test_grouped = df_test.groupby(["tollgate_id", "direction"])
result = []
oob = []
for key, train_data in df_train_grouped:

    test_data = df_test_grouped.get_group(key)
    len_train = len(train_data)
    train_data = train_data.append(test_data)[train_data.columns.tolist()]
    train_data = feature_transform_split_complete(key, train_data)

    regressor_cubic = RandomForestRegressor(n_estimators=500, max_features='sqrt', random_state=10, oob_score=True)
    train_data = pd.DataFrame.reset_index(train_data)
    train_data = train_data.drop("index", axis=1)
    y = train_data.ix[:len_train - 1, :]["volume"]
    x = train_data.ix[:len_train - 1, 8:]
    x1 = train_data.ix[len_train:, 8:]
    regressor_cubic.fit(x, y)
    yhat = regressor_cubic.predict(x1)


/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.p

In [147]:
len(yhat)
len(values_test)

10065

In [85]:
regressor_cubic.fit(feature_train[['tollgate_id', 'direction', 'weekday', 'weekend', 'hour', 'minute', 'holiday', 'am_pm', 'rel_humidity']], values_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=True, random_state=10, verbose=0, warm_start=False)

In [86]:
y_pred = regressor_cubic.predict(feature_test[['tollgate_id', 'direction', 'weekday', 'weekend', 'hour', 'minute', 'holiday', 'am_pm', 'rel_humidity']])

In [178]:
df_value_pred = pd.read_csv('data_final/resultado_final.csv')
df_value_real = pd.read_csv('../dataset/test2_20min_avg_volume_test.csv')
values_pred = df_value_pred['volume'].values
values_reais = df_value_real['volume'].values
len(values_reais)

420

In [89]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [179]:
mean_absolute_percentage_error(values_reais, values_pred)

122.39570726692179

In [116]:
mape

0.8089286811859463

In [93]:
len(y_pred)

1120

In [94]:
len(values_test)

1120

In [180]:
mape = np.mean(np.abs((values_pred - values_reais)/values_reais))

In [181]:
mape

1.223957072669218

In [182]:
values_pred

array([ 66.634     ,  68.29      ,  62.234     ,  61.974     ,
        61.382     ,  61.236     ,  59.426     ,  61.052     ,
        54.99      ,  52.664     ,  52.722     ,  53.07      ,
        66.634     ,  68.29      ,  62.234     ,  61.974     ,
        61.382     ,  61.236     ,  59.426     ,  61.052     ,
        54.99      ,  52.664     ,  52.722     ,  53.07      ,
        66.634     ,  68.29      ,  62.234     ,  61.974     ,
        61.382     ,  61.236     ,  59.426     ,  61.052     ,
        54.99      ,  52.664     ,  52.722     ,  53.07      ,
        66.634     ,  68.29      ,  62.234     ,  61.974     ,
        61.382     ,  61.236     ,  59.426     ,  61.052     ,
        54.99      ,  52.664     ,  52.722     ,  53.07      ,
        69.892     ,  72.118     ,  70.818     ,  70.156     ,
        69.8       ,  70.364     ,  63.45      ,  64.942     ,
        62.11      ,  61.938     ,  61.48      ,  62.884     ,
        69.892     ,  72.118     ,  70.818     ,  70.15

In [184]:
values_reais

array([ 28,  30,  29,  10,  37,  48,  31,  27,  17,  54,  92,  42,  30,
        63,  40,  83,  25,  67, 129,  62,  98, 131,  89,  38,  99,  93,
       139,  84,  44, 133, 125,  79,  77,  52,  92,  44, 111, 123, 101,
        66, 124,  81, 100,  46,  89, 126,  96,  78,  50, 113,  83, 141,
       104,  40, 107, 118,  98,  50, 108,  69,  32,  12,  37,  24,  21,
        50,  36,  27,   9,  51,  74,  41,  25,  57,  39,  74, 129,  50,
        31,  69,  88, 121,  82,  32,  96, 144,  86,  99,  45, 110, 120,
        81,  38,  96,  72,  59, 107,  80,  36,  99, 114,  79,  74,  44,
       101, 131,  83,  68,  49, 110,  80, 132,  93,  45, 113,  90, 127,
       122,  46, 109,  14,  37,  20,  24,  24,  40,  25,  36,  14,  46,
        63,  49,  50,  23,  51, 120,  52,  70,  20,  62, 112,  90,  81,
        36,  94, 134,  86,  42, 104, 114, 110, 100,  58,  40,  87,  54,
        93,  89,  37, 101, 105,  80,  45, 101,  79,  49,  96, 113, 107,
        57, 126, 108,  73,  41,  93,  83, 125, 117,  45, 109,  2